In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
import numpy as np

dataset_fname = "QSAR_dataset.xlsx"
test_fname = "test_TP1.xlsx"


# Représentation des données
Dans cette étape, nous devons:
- analyser chaque attribut;
- proposer un prétraitement des données;
- sélectionner les 10 meilleurs attributs avec justification statistique;
- visualiser la distribution des 10 meilleurs attributs.

### Chargement des données

In [ ]:
# Chargement des données d'entraînement
df = pd.read_excel(dataset_fname, index_col=0)
# Lecture rapide des cinq premières entrées pour valider le chargement adéquat des données
df.head(5)

## Analyser chaque attribut

#### Attributs quantitatifs et qualitatifs
D'abord, commençons par distinguer les attributs qualitatifs des attributs numériques. Nous portons une attention particulière aux attributs qualitatifs (ou catégoriques) car ceux-ci nécessitent du pré-traitement particulier.

In [ ]:
df.info()

In [ ]:
num_cols = df.select_dtypes(exclude=["object", "category"]).columns.tolist()
cat_cols = df.select_dtypes(include=["category", "object"]).columns.tolist()
print("Nombre d'attributs numériques: {}".format(len(num_cols)))
print("Nombre d'attributs catégoriques: {}".format(len(cat_cols)))
if len(cat_cols) > 0:
    print(cat_cols)

Seul l'attribut `Class` est catégorique, ce qui est normal puisqu'il associe une classe à chacune des observations. Tous nos attributs sont donc numériques, ce qui facilitera grandement notre prétraitement.

#### Valeurs aberrantes
Maintenant, détectons les valeurs abberantes ou invalides (`NaN`, `±inf`, etc.). Dans notre traitement, nous commençons par convertir les valeurs `±inf` en `NaN` afin d'éviter des duplicata de code. En effet, ces valeurs nécessiteront le même pré-traitement alors une stratégie optimale consiste à les considérer de la même façon dès le départ.

In [ ]:
df = df.replace([-np.inf, np.inf], np.nan)
nan_cols = df.columns[df.isna().any()]
nan_summary = df[nan_cols].isna().sum().sort_values(ascending=False)
nan_ratio = nan_summary / len(df)
nan_df = pd.concat((nan_summary, nan_ratio), axis=1)
nan_df = nan_df.rename(columns={0: "Count", 1: "Ratio"})
print(nan_df)

On constate que les variables `vsurf_V` et `vsurf_S` contiennent beaucoup de valeurs NaN (11.69% et 8.44% respectivement). Par contraste, `ASA+`, `ASA-`, `a_heavy` et `a_IC` contiennent seulement entre 1 et 2 observations invalides. Plus tard, nous pourrons probablement les supprimer sans affecter grandement la distribution des données.

#### Valeurs uniques
Finalement, nous devons considérons les valeurs uniques. Celles-ci ne sont pas techniquement invalides, mais peuvent néamoins être ignorées car elles ne contribueront pas à la décision des algorithmes. En effet, puisque ces attributs prennent une seule valeur distincte, elles ne seront pas déterminantes dans la tâche de classification.

In [ ]:
uniq_cols = df.columns[df.nunique() == 1]
uniq_cols

Tous les attributs possèdent donc plusieurs valeurs distinctes.

#### Analyse statistique
On peut maintenant extraire certaines statistiques de base pour tous les attributs

In [ ]:
df_describe = df.describe()
df_describe

On normalise les différentes variances en fonction de leur moyenne respective. Ceci permet d'évaluer le degré de dispersion des différents attributs sous une même échelle. Des grandes moyennes vont naturellement générées de grandes variances mais cela n'implique pas nécessairement que les données sont très dispersées; l'échelle de mesure est simplement grande. 

In [ ]:
np_std = df_describe.loc[["std"]].to_numpy()
np_mean = df_describe.loc[["mean"]].to_numpy()
normalized_std = np_std / np.abs(np_mean)
normalized_describe = df_describe.copy()
normalized_describe.loc[["std"]] = normalized_std
normalized_describe.sort_values(by="std", ascending=False, axis=1)

On constate une très grande variance au niveau de `vsurf_R` et une variance presque nulle pour `VAdjMa`. On peut donc déjà indiqué que `VAdjMa` présente une distribution centrée autour de sa moyenne que `vsurf_R` sera très dispersée. On peut s'en convaincre par une visualisation. Cette dernière variable est donc plus "bruitée" mais en même temps contient plus d'information que la première. Ce constat pourrait être utile lors de la sélection des attributs plus tard.

In [ ]:
f, ax = plt.subplots(figsize=(12, 9))
sns.scatterplot(x=df["VAdjMa"], y=df["VAdjMa"], hue=df["Class"])


#### Analyse de la covariance
On peut maintenant évaluer la matrice de variance-covariance entre les attributs. 

In [ ]:
# Code inspiré de https://seaborn.pydata.org/examples/many_pairwise_correlations.html
# Pour éviter de la confusion, on calcule la matrice de covariance absolue.
# Ainsi, les cases bleues indiquent une absence de corrélation et les cases rouges l'inverse.
corr_mat = df[num_cols].corr().abs()
# On masque le haut de la matrice diagonale
mask = np.triu(np.ones_like(corr_mat, dtype=bool))
f, ax = plt.subplots(figsize=(30, 20))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr_mat, annot=False, cmap=cmap)

In [ ]:
# Code inspiré de https://chrisalbon.com/code/machine_learning/feature_selection/drop_highly_correlated_features/
# Nous avons testé plusieurs ratio différents et celui-ci permettait de filtrer plusieurs attributs.
max_rho = 0.75
# Sélectionner triangle supérieur de la matrice de corrélation
upper = corr_mat.where(np.triu(np.ones_like(corr_mat), k=1).astype(bool))
# Sélectionner les attributs dont la corrélation dépasse le seuil `max_rho`
to_drop = [column for column in upper.columns if any(upper[column] >= max_rho)]
print("Total d'attributs dépassant le seuil {:.2f}: {}".format(max_rho, len(to_drop)))
print(to_drop)

In [ ]:
df_prime = df.drop(columns=to_drop, inplace=False)
print("Nombre d'attributs supprimés: {}".format(len(to_drop)))
print("Ratio : {:8.4f}".format(len(to_drop) / len(df)))
print("Nombre d'attributs restants: {}".format(len(df_prime.columns)))

In [ ]:
num_cols_prime = df_prime.select_dtypes(exclude=["object", "category"]).columns.tolist()
corr_mat_prime = df_prime[num_cols_prime].corr().abs()
# On masque le haut de la matrice diagonale
f, ax = plt.subplots(figsize=(30, 20))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr_mat_prime, annot=False, cmap=cmap)